#### 불균형 클래스
#### 클래스 불균형에 대한 이해
#### 클래스 가중치 사용
#### 리샘플링 기법
#### 적절한 평가지표

In [ ]:
# 불균형 데이터 생성(1:9)  악성: 생성
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.pipeline import Pipeline
np.random.seed(42) #np이용한 데이터 바뀌어서 42로 고정 np쓰기전에 고정 
data = load_breast_cancer()
X,y = data.data, data.target
#악성을 소수 클래스로 생성
#pandasdml value_counts역할하는거 np.unique
print(f' 악성 양성의 오리지널 비율 : {np.unique(y,return_counts=True)}')
m_index = np.where(y == 0)[0] #where은 필터링 0이 되는index를 반환 index위치에 해당하는 x 가지고 오기 위해서 #index
b_index = np.where(y == 1)[0]

#악성은 일부만, 양성은 더 많이 사용
# 악성의 30%만 사용
size_30 = int(len(m_index) *0.1)
selected_m_index = np.random.choice(m_index, size=size_30,replace = False) # 임의로 랜덤하게 뽑아주는거 
selected_b_index = b_index

concatenate_selected_index = np.concatenate([selected_m_index,selected_b_index])
np.random.shuffle(concatenate_selected_index) #return값 없다 출력하면 바뀜

X_imb =X[concatenate_selected_index]
y_imb =y[concatenate_selected_index]

#클래스 분포 확인
unique,counts = np.unique(y_imb,return_counts = True)
print('클래스 분포')
for label, count in zip(unique, counts):
    percentage = count / len(y_imb) *100
    print(f'클래스 {label} : {count}개 ({percentage:.1f}%)')

# 불균형인 상태로 진행
# 스케일링 정규화 StandardScaler
# LogisticRegression
# pipe
# 평가는 class report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train,X_test,y_train,y_test = train_test_split(X_imb,y_imb,stratify=y_imb, test_size=0.2, random_state=42)
print('1. 기본모델(불균형무시)')
pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', LogisticRegression(random_state=42,max_iter=1000))
])
pipe.fit(X_train,y_train)
print(classification_report(y_test, pipe.predict(X_test),target_names=['악성(0)','양성(1)']))

 악성 양성의 오리지널 비율 : (array([0, 1]), array([212, 357]))
클래스 분포
클래스 0 : 21개 (5.6%)
클래스 1 : 357개 (94.4%)
1. 기본모델(불균형무시)
              precision    recall  f1-score   support

       악성(0)       1.00      1.00      1.00         4
       양성(1)       1.00      1.00      1.00        72

    accuracy                           1.00        76
   macro avg       1.00      1.00      1.00        76
weighted avg       1.00      1.00      1.00        76



In [32]:
print('불균형 해결 : 클래스 가중치 사용')
# 기존파이프라인의 clf이름의 객체의 파라메터를 조정
from copy import deepcopy
pipe_weight = deepcopy(pipe)
pipe_weight.set_params(clf__class_weight = 'balanced')
pipe_weight.fit(X_train, y_train)
print(classification_report(y_test,pipe_weight.predict(X_test)))

print("가중치 계산")
n_samples = len(y_train)
n_classes = 2
class_counts = np.bincount(y_train)
for i in range(n_classes):
    weight = n_samples / (n_classes * class_counts[i])
    print(f'클래스: {i} : {weight:.3f}')

불균형 해결 : 클래스 가중치 사용
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         4
           1       1.00      0.99      0.99        72

    accuracy                           0.99        76
   macro avg       0.90      0.99      0.94        76
weighted avg       0.99      0.99      0.99        76

가중치 계산
클래스: 0 : 8.882
클래스: 1 : 0.530


In [35]:
from sklearn.ensemble import RandomForestClassifier
print('불균형: 랜덤포레스트( 균형모드)')
pipe_rf = Pipeline([
    ('scaler',StandardScaler()),
    ('clf',RandomForestClassifier(class_weight='balanced',random_state=42))
])
pipe_rf.fit(X_train, y_train)
y_pred = pipe_rf.predict(X_test)
print(classification_report(y_test,y_pred))

불균형: 랜덤포레스트( 균형모드)
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         4
           1       0.97      1.00      0.99        72

    accuracy                           0.97        76
   macro avg       0.99      0.75      0.83        76
weighted avg       0.97      0.97      0.97        76



In [ ]:
# 오버 or 언더샘플링 - 직접 실행.. SMOTE SMOTEEN 과적합이 일어나고 있는데 해결
